# Детекция на примере `yolov5`

## Подготовка

Клонируем репозиторий [`ultralytics/yolov5`](https://github.com/ultralytics/yolov5)

In [17]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 16531, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 16531 (delta 1), reused 5 (delta 0), pack-reused 16522
Receiving objects: 100% (16531/16531), 15.05 MiB | 30.29 MiB/s, done.
Resolving deltas: 100% (11354/11354), done.


In [18]:
%cd yolov5

/kaggle/working/yolov5/yolov5/yolov5


In [19]:
%ls

CITATION.cff     README.zh-CN.md  detect.py   pyproject.toml    tutorial.ipynb
CONTRIBUTING.md  benchmarks.py    export.py   requirements.txt  utils/
LICENSE          classify/        hubconf.py  segment/          val.py
README.md        data/            models/     train.py


In [20]:
!python -m pip install -r requirements.txt

## Пробуем запустить

Сначала попробуем запустить код с их обученной моделью, а потом будем пробовать запускать своё обучение

In [21]:
# %cat data/Objects365.yaml

In [22]:
# %ls data/Objects365.yaml

In [23]:
!python val.py --weights yolov5m.pt --data data/Objects365.yaml --imgsz 640 --device 0  # cuda:0

val: data=data/Objects365.yaml, weights=['yolov5m.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=0, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-298-g21f8f94d Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)

100%|███████████████████████████████████████| 40.8M/40.8M [00:00<00:00, 305MB/s]

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients, 48.9 GFLOPs

Dataset not found ⚠️, missing paths ['/kaggle/working/yolov5/yolov5/datasets/Objects365/images/val']
Processing train in 51 patches ...
100%|██████████████████████████████████████| 1.24G/1.24G [02:02<00:00, 10.9MB/s]
Unzipping /kaggle/working/yolov5/yolov5/datasets/Objects365/zhiyuan_objv2_train.tar.gz...
tar: zhiyuan_objv2_train.json: Wrote only 6656 of 10240 bytes
tar: Exiting with failure sta

In [24]:
%ls ../datasets/Objects365

images/  labels/  zhiyuan_objv2_train.json*  zhiyuan_objv2_train.tar.gz


In [25]:
# %ls ../datasets/coco128/images/train2017

In [26]:
# %ls ../datasets/coco128/labels/train2017/

Посмотрим на то, что представляют из себя файлы с аннотациями

In [27]:
# %cat  ../datasets/coco128/labels/train2017/000000000009.txt

Формат разметки, который здесь используется, называется `yolo`-форматом:

`[cls, x_center / img_w, y_center / img_h, w / img_w, h / img_h]`

Есть и другие популярные форматы разметки, например, `coco`-формат. А вот что значат цифры в начале:

```text
45: bowl
46: banana
47: apple
48: sandwich
49: orange
50: broccoli
```

In [29]:
import cv2
import matplotlib.pyplot as plt
# from google.colab.patches import cv2_imshow

In [30]:
# img = cv2.imread('../datasets/coco128/images/train2017/000000000009.jpg')
# cv2_imshow(img)
# plt.show()

In [31]:
# !python detect.py --weights yolov5m.pt --source ../datasets/coco128/images/train2017/000000000009.jpg --name some_name

In [32]:
# img = cv2.imread('runs/detect/some_name/000000000009.jpg')
# cv2_imshow(img)
# plt.show()

Также можно запустить `detect.py` не только на отдельном изображении, но и на папке с изображениями, на потоке, на изображении из сети и т.п.

## Как запустить своё обучение

Чтобы запустить своё обучение, надо:

- Подготовить датасет в `yolo`-формате, создать свой конфиг с путями до данных вида `data/my_data.yaml`. Примеры того, как организовывать файлы в датасет, можно посмотреть в [`data/coco.yaml`](https://github.com/ultralytics/yolov5/blob/master/data/coco.yaml) и [`data/VOC.yaml`](https://github.com/ultralytics/yolov5/blob/master/data/VOC.yaml)
- Создать файлик с конфигом, в котором будут прописаны параметры для тренировки, вида `data/hyps/my_hyps.yaml`. Для начала можно попробовать запустить со стандартным [конфигом](https://github.com/ultralytics/yolov5/blob/master/data/hyps/hyp.scratch-low.yaml)
- Стартовать с весов `yolov5s`, если будете тренировать yolo small; `yolov5n`, если yolo nano, и т.д.
![](https://github.com/ultralytics/yolov5/releases/download/v1.0/model_comparison.png)
- Лучше передавать в `--name` какое-то адекватное название, которое описывает суть эксперимента, чтобы не перепутать потом их между собой
- Графики можно смотреть во время обучения, для этого надо запустить в терминале что-то типа:

  ` tensorboard --logdir ./runs/train/`

  и перейти по ссылке в браузере

Пример:

In [41]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="vVzgCl7OcCV2hguH2FcC")
project = rf.workspace("cs474-ug2-vehicle-detection").project("object-detection-um7ee")
version = project.version(3)
dataset = version.download("yolov5")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Object-Detection-3 in yolov5pytorch:: 100%|██████████| 7882/7882 [00:02<00:00, 3920.23it/s]


In [ ]:
%cd yolov5
!ls

In [43]:
!python train.py --data Object-Detection-3/data.yaml --hyp data/hyps/hyp.scratch-high.yaml --weights yolov5m.pt \
--epochs 50 --batch-size 64 --optimizer SGD --name lab2_small  --imgsz 1500

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-04-13 18:27:46.069423: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-13 18:27:46.069494: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-13 18:27:46.071002: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train:

In [ ]:
# !zip -r /content/file.zip /content/yolov5/runs/train/lab2_small

Какие ещё ключи есть у `train.py`? Идём в скрипт и находим вот такую часть кода, читаем её:

```python
def parse_opt(known=False):
    parser = argparse.ArgumentParser()
    parser.add_argument('--weights', type=str, default=ROOT / 'yolov5s.pt', help='initial weights path')
    parser.add_argument('--cfg', type=str, default='', help='model.yaml path')
    parser.add_argument('--data', type=str, default=ROOT / 'data/coco128.yaml', help='dataset.yaml path')
    parser.add_argument('--hyp', type=str, default=ROOT / 'data/hyps/hyp.scratch-low.yaml', help='hyperparameters path')
    parser.add_argument('--epochs', type=int, default=100, help='total training epochs')
    parser.add_argument('--batch-size', type=int, default=16, help='total batch size for all GPUs, -1 for autobatch')
    parser.add_argument('--imgsz', '--img', '--img-size', type=int, default=640, help='train, val image size (pixels)')
    parser.add_argument('--rect', action='store_true', help='rectangular training')
    parser.add_argument('--resume', nargs='?', const=True, default=False, help='resume most recent training')
    parser.add_argument('--nosave', action='store_true', help='only save final checkpoint')
    parser.add_argument('--noval', action='store_true', help='only validate final epoch')
    parser.add_argument('--noautoanchor', action='store_true', help='disable AutoAnchor')
    parser.add_argument('--noplots', action='store_true', help='save no plot files')
    parser.add_argument('--evolve', type=int, nargs='?', const=300, help='evolve hyperparameters for x generations')
    parser.add_argument('--bucket', type=str, default='', help='gsutil bucket')
    parser.add_argument('--cache', type=str, nargs='?', const='ram', help='image --cache ram/disk')
    parser.add_argument('--image-weights', action='store_true', help='use weighted image selection for training')
    parser.add_argument('--device', default='', help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
    parser.add_argument('--multi-scale', action='store_true', help='vary img-size +/- 50%%')
    parser.add_argument('--single-cls', action='store_true', help='train multi-class data as single-class')
    parser.add_argument('--optimizer', type=str, choices=['SGD', 'Adam', 'AdamW'], default='SGD', help='optimizer')
    parser.add_argument('--sync-bn', action='store_true', help='use SyncBatchNorm, only available in DDP mode')
    parser.add_argument('--workers', type=int, default=8, help='max dataloader workers (per RANK in DDP mode)')
    parser.add_argument('--project', default=ROOT / 'runs/train', help='save to project/name')
    parser.add_argument('--name', default='exp', help='save to project/name')
    parser.add_argument('--exist-ok', action='store_true', help='existing project/name ok, do not increment')
    parser.add_argument('--quad', action='store_true', help='quad dataloader')
    parser.add_argument('--cos-lr', action='store_true', help='cosine LR scheduler')
    parser.add_argument('--label-smoothing', type=float, default=0.0, help='Label smoothing epsilon')
    parser.add_argument('--patience', type=int, default=100, help='EarlyStopping patience (epochs without improvement)')
    parser.add_argument('--freeze', nargs='+', type=int, default=[0], help='Freeze layers: backbone=10, first3=0 1 2')
    parser.add_argument('--save-period', type=int, default=-1, help='Save checkpoint every x epochs (disabled if < 1)')
    parser.add_argument('--seed', type=int, default=0, help='Global training seed')
    parser.add_argument('--local_rank', type=int, default=-1, help='Automatic DDP Multi-GPU argument, do not modify')

    # Logger arguments
    parser.add_argument('--entity', default=None, help='Entity')
    parser.add_argument('--upload_dataset', nargs='?', const=True, default=False, help='Upload data, "val" option')
    parser.add_argument('--bbox_interval', type=int, default=-1, help='Set bounding-box image logging interval')
    parser.add_argument('--artifact_alias', type=str, default='latest', help='Version of dataset artifact to use')

    return parser.parse_known_args()[0] if known else parser.parse_args()
```



(10 эпох - это достаточно мало, да и сама эпоха у вас будет занимать гораздо больше времени, ведь coco128 - это маленький датасет для отладки)

Приятная опция: прерванные эксперименты можно возобновлять (если нечаянно нажали ctrl + c или потерялось соединение с сервером). Так выглядит код повторного запуска обучения, если вы прервали его:

In [ ]:
# !python train.py --data data/Objects365.yaml --hyp data/hyps/hyp.scratch-low.yaml --weights yolov5m.pt \
# --epochs 5 --batch-size 32 --optimizer SGD --name example_Objects365 --resume runs/train/example_Objects365/weight/last.pt

Провалидировать модель можно при помощи val.py:

In [ ]:
!ls

In [ ]:
# !python val.py --weights best.pt --data Object-Detection-3/data.yaml

## Если остались вопросы

- Читаем скрипты в [репозитории](https://github.com/ultralytics/yolov5)
- Читаем [документацию к репозиторию](https://github.com/ultralytics/yolov5#documentation)
- Если остались вопросы, можно искать в [issues](https://github.com/ultralytics/yolov5/issues)

In [ ]:
!ls